In [1]:
import tensorflow as tf 
print(tf.__version__)

1.1.0


In [2]:
#loading modules and  packages
import logging
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import datetime
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.externals import joblib
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core import Experiment
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage
from azureml.core.webservice import AciWebservice
from azureml.core.conda_dependencies import CondaDependencies
import collections
from sklearn.model_selection import train_test_split
import pickle as pkl
from sklearn.linear_model import Ridge
from azureml.core.model import Model

In [3]:
ws = Workspace.from_config()

In [12]:
print(ws)

Workspace.create(name='xm-ml-workspace', subscription_id='be564fde-136b-4709-b7b6-abfc0bdfc134', resource_group='xm-ml')


In [5]:
#registering the model
model = Model.register(model_name = "fraudclassifyv2",
                       model_path = "version2/riskmodel.pkl",
                       tags = {"key": "3","classification": "fraud","version":"2"},
                       description = "riskpredictanalysis",
                       workspace = ws)

Registering model fraudclassifyv2


In [6]:
print(model.name)

fraudclassifyv2


In [7]:
%%writefile score.py
import json
import pickle
import numpy as np
import pandas as pd
import azureml.train.automl
from sklearn.externals import joblib
from azureml.core.model import Model

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType
from inference_schema.parameter_types.pandas_parameter_type import PandasParameterType


input_sample = pd.DataFrame(data=[{"TOTAL_PRICE":869.92,"THIRD_PARTY_ID_SCORE":415,"IS_EXISTING_CUSTOMER":"Y","NUM_PORTIN":0,"FIRST_PARTY_ID_SCORE":356,"ONETIMECHARGE":33.33,"INSTALLMENT_AMOUNT":33.33,"DEVICE_AT_HOME":"Y","FRAUDNET_SCORE":200.0,"NUM_BYOD":0,"IDA_RESULT":"GREEN","EXTERNAL_CREDIT_CHECK_DONE":"Y","SALES_CHANNEL":"ONLINE","MODEL1":"iPhone XR","MODEL2":"NA","MODEL3":"NA","MODEL4":"NA","MODEL5":"NA"}])
output_sample = np.array([0])


def init():
    global model
    # This name is model.id of model that we want to deploy deserialize the model file back
    # into a sklearn model
    model_path = Model.get_model_path(model_name = 'fraudclassifyv2')
    model = joblib.load(model_path)

@input_schema('data', PandasParameterType(input_sample))
@output_schema(NumpyParameterType(output_sample))
def run(data):
    try:
        result = model.predict(data)
        if result ==0:
            result = 'GREEN'
            print('GREEN')
        elif result ==1:
            result = 'YELLOW'
            print('YELLOW')
        else:
            result = 'RED'
            print('RED')
        print('result') 
        fraud_status =[]
        fraud_status.append(result)
        return fraud_status
    except Exception as e:
        result = str(e)
        return json.dumps({"error": result})
    return json.dumps({"result": result.tolist()})

Overwriting score.py


In [8]:
#create environment file
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import Model
import numpy as np
import pandas as pd
from sklearn import datasets
import datetime
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.externals import joblib
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core import Experiment
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage
from azureml.core.webservice import AciWebservice
from azureml.core.conda_dependencies import CondaDependencies
import collections
from sklearn.model_selection import train_test_split
import pickle as pkl
fraudenv = CondaDependencies()
fraudenv.add_conda_package("scikit-learn")
fraudenv.add_conda_package("numpy")
fraudenv.add_conda_package("scipy")
fraudenv.add_conda_package("pandas")
fraudenv.add_conda_package("py-xgboost<=0.80")
fraudenv.add_pip_package("azureml-defaults")
fraudenv.add_pip_package("azureml-train-automl==1.0.55")
fraudenv.add_pip_package("azureml-core==1.0.55")
fraudenv.add_pip_package("inference-schema")
fraudenv.conda_dependencies_file_path="data/env_dependencies.json"
with open("riskpredictionenv.yml","w") as f:
    f.write(fraudenv.serialize_to_string())

In [13]:
from azureml.core.compute import AksCompute, ComputeTarget
aks_name = 'xm-ml-er-aks2'
aks_target = AksCompute(ws, aks_name)
print(aks_name)

xm-ml-er-aks2


In [11]:
# creating a container image
from azureml.core.image import Image, ContainerImage
image_config = ContainerImage.image_configuration(runtime= "python",
                                 execution_script="score.py",
                                 conda_file="riskpredictionenv.yml",
                                 tags = {"key": "1","image": "riskpredictanalysis"},
                                 description = "fraudclassifyv2")
image = Image.create(name = "fraudclassification",
                     # this is the model object 
                     models = [model],
                     image_config = image_config, 
                     workspace = ws)
image.wait_for_creation(show_output = True)

Creating image
Running..............................................................
Succeeded
Image creation operation finished for image fraudclassification:3, operation "Succeeded"


In [14]:
#create inference config
from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(runtime= "python", 
                                   entry_script="score.py",
                                   conda_file="riskpredictionenv.yml")

In [16]:
#service deployment
from azureml.core.webservice import AksWebservice, Webservice
from azureml.core.model import Model
from azureml.core.model import InferenceConfig
from azureml.core.image import Image, ContainerImage


aks_target = AksCompute(ws,"xm-ml-er-aks2")

# If deploying to a cluster configured for dev/test, ensure that it was created with enough
# cores and memory to handle this deployment configuration. Note that memory is also used by
# things such as dependencies and AML components.
deployment_config = AksWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
service = Model.deploy(ws, "fraudclassifyv2", [model] , inference_config, deployment_config, aks_target)
service.wait_for_deployment(show_output = True)
print(service.state)
print(service.get_logs())

Running...................
Failed

ERROR - Service deployment polling reached non-successful terminal state, current service state: Failed
More information can be found using '.get_logs()'
Error:
{
  "code": "KubernetesDeploymentFailed",
  "statusCode": 400,
  "message": "Kubernetes Deployment failed",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs for your container instance: fraudclassifyv2. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs. \nYou can also try to run image xmmlworkspac13b1f937.azurecr.io/azureml/azureml_80ae16a42256ddf5cb3fe42530de07e1:latest locally. Please refer to http://aka.ms/debugimage#service-launch-fails for more information."
    }
  ]
}

ERROR - Service deployment polling reached non-successful terminal state, current service state: Failed
More information can be found using '.get_logs()'
E

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
More information can be found using '.get_logs()'
Error:
{
  "code": "KubernetesDeploymentFailed",
  "statusCode": 400,
  "message": "Kubernetes Deployment failed",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs for your container instance: fraudclassifyv2. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs. \nYou can also try to run image xmmlworkspac13b1f937.azurecr.io/azureml/azureml_80ae16a42256ddf5cb3fe42530de07e1:latest locally. Please refer to http://aka.ms/debugimage#service-launch-fails for more information."
    }
  ]
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Service deployment polling reached non

In [25]:
print(service.scoring_uri)

http://13.67.176.88:80/api/v1/service/fraudclassify/score


In [26]:
primary, secondary = service.get_keys()
print(primary)

qLEcUK30NCjceE6Klg2mnceNTfsNHZT8
